In [144]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [145]:
original_df = pd.read_csv("data/stackoverflow_users_all_notext.csv",nrows=20000)

In [146]:
"""Show the data."""
display(original_df[130:140])

,id,age,location,creation_date,reputation,up_votes,down_votes
130,4845736,NaN,NaN,2015-04-29 09:41:03.96 UTC,1,0,0
131,4890609,NaN,"Belo Horizonte - State of Minas Gerais, Brazil",2015-05-12 09:03:06.03 UTC,1,0,0
132,4965490,NaN,"Talca, Chile",2015-06-02 13:36:06.94 UTC,1,0,0
133,5006338,NaN,NaN,2015-06-13 14:17:20.823 UTC,1,0,0
134,5073970,NaN,NaN,2015-07-02 13:15:12.983 UTC,1,0,0
135,5150946,NaN,"Melbourne, Australia",2015-07-24 05:58:04.17 UTC,1,0,0
136,5185772,NaN,NaN,2015-08-03 13:10:44.243 UTC,1,0,0
137,5231705,NaN,jakarta,2015-08-16 06:01:09.437 UTC,1,0,0
138,5248405,NaN,NaN,2015-08-20 16:12:00.633 UTC,1,0,0
139,5281835,NaN,NaN,2015-08-30 12:45:03.383 UTC,1,0,0


The location field seems to have the country name as the last part quite consistently, separated 
by a comma and space. Some of the countries are, however, named using some non-English form. 
Let's download a dataset of country names in different languages and use them to match locations to countries.

In [147]:
clean_df = original_df[original_df.location.notnull()]
notinteresting_df = original_df[original_df.location.isnull()]

In [148]:
"""Extract the country field out of the location field"""
clean_df["country"] = clean_df["location"].apply(lambda x: x.split(", ")[-1].lower() if isinstance(x,str) else np.nan)

In [149]:
clean_df.head(5)

,id,age,location,creation_date,reputation,up_votes,down_votes,country
1,7019478,NaN,Singapore,2016-10-14 13:36:30.96 UTC,1,0,0,singapore
2,7074992,NaN,Egypt,2016-10-26 12:11:08.41 UTC,1,0,0,egypt
3,7093392,NaN,"New York, NY, United States",2016-10-31 02:19:04.64 UTC,1,0,0,united states
6,7395536,NaN,"Cluj-Napoca, Cluj County, Romania",2017-01-09 16:36:43.747 UTC,1,0,0,romania
8,7497308,NaN,"Bodø, Norge",2017-01-31 21:13:26.053 UTC,1,0,0,norge


In [150]:
"""Load the dataset of country names in different languages got from ip2location.com."""
country_names = pd.read_csv("data/IP2LOCATION-COUNTRY-MULTILINGUAL/IP2LOCATION-COUNTRY-MULTILINGUAL.CSV")

In [151]:
display(country_names.head())

,LANG,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_ALPHA3_CODE,COUNTRY_NUMERIC_CODE,COUNTRY_NAME
0,AF,AFRIKAANS,AD,AND,20,Andorra
1,AF,AFRIKAANS,AE,ARE,784,Verenigde Arabiese Emirate
2,AF,AFRIKAANS,AF,AFG,4,Afganistan
3,AF,AFRIKAANS,AG,ATG,28,Antigua en Barbuda
4,AF,AFRIKAANS,AI,AIA,660,Anguilla


In [152]:
"""Pick only the relevant part of the data"""
country_name_to_code = country_names[['COUNTRY_NAME','COUNTRY_ALPHA3_CODE']]
country_name_to_code.COUNTRY_NAME = country_name_to_code.COUNTRY_NAME.str.lower()
"""Drop duplicates since many (or probably all) countries have the same name in several languages"""
country_name_to_code = country_name_to_code[~country_name_to_code.COUNTRY_NAME.duplicated()]

In [153]:
clean_df = clean_df.merge(country_name_to_code, left_on="country", right_on="COUNTRY_NAME", how="left")
clean_df.drop(["COUNTRY_NAME"], axis=1, inplace=True)

In [154]:
clean_df.head()

,id,age,location,creation_date,reputation,up_votes,down_votes,country,COUNTRY_ALPHA3_CODE
0,7019478,NaN,Singapore,2016-10-14 13:36:30.96 UTC,1,0,0,singapore,SGP
1,7074992,NaN,Egypt,2016-10-26 12:11:08.41 UTC,1,0,0,egypt,EGY
2,7093392,NaN,"New York, NY, United States",2016-10-31 02:19:04.64 UTC,1,0,0,united states,USA
3,7395536,NaN,"Cluj-Napoca, Cluj County, Romania",2017-01-09 16:36:43.747 UTC,1,0,0,romania,ROU
4,7497308,NaN,"Bodø, Norge",2017-01-31 21:13:26.053 UTC,1,0,0,norge,NOR


In [155]:
print("Proportion of matches: ", sum(clean_df.COUNTRY_ALPHA3_CODE.notnull())/sum(clean_df.location.notnull()))

Proportion of matches:  0.8235167206040992


The method manages to add a country tag to 86.9% of the users who have informed a country in the first place. Let's then look at the users that weren't matched.

In [156]:
clean_df[clean_df.location.notnull() & clean_df.COUNTRY_ALPHA3_CODE.isnull()].head()

,id,age,location,creation_date,reputation,up_votes,down_votes,country,COUNTRY_ALPHA3_CODE
6,7579333,NaN,"Beijing, 北京市中国",2017-02-17 07:27:20.823 UTC,1,0,0,北京市中国,NaN
13,8448975,NaN,Bangalore,2017-08-11 05:32:54.82 UTC,1,0,0,bangalore,NaN
15,8623520,NaN,deverloper,2017-09-17 19:52:14.057 UTC,1,0,0,deverloper,NaN
20,9569837,NaN,UK,2018-03-29 12:56:51.147 UTC,1,0,0,uk,NaN
22,9867362,NaN,"Hangzhou, 浙江省 China",2018-05-30 01:06:04.59 UTC,1,0,0,浙江省 china,NaN


Seems like a part of the problem is people giving out only the city or some funny stuff. Let's try addressing this by loading a dataset of city names and trying to match them with the remaining, unmatched data.

In [157]:
world_cities = pd.read_csv("data/worldcities.csv")
display(world_cities.head())

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Malishevë,Malisheve,42.4822,20.7458,Kosovo,XK,XKS,Malishevë,admin,NaN,1901597212
1,Prizren,Prizren,42.2139,20.7397,Kosovo,XK,XKS,Prizren,admin,NaN,1901360309
2,Zubin Potok,Zubin Potok,42.9144,20.6897,Kosovo,XK,XKS,Zubin Potok,admin,NaN,1901608808
3,Kamenicë,Kamenice,42.5781,21.5803,Kosovo,XK,XKS,Kamenicë,admin,NaN,1901851592
4,Viti,Viti,42.3214,21.3583,Kosovo,XK,XKS,Viti,admin,NaN,1901328795


In [158]:
display(world_cities[world_cities.city=="Moscow"])

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
2220,Moscow,Moscow,55.7522,37.6155,Russia,RU,RUS,Moskva,primary,10452000.0,1643318494
11641,Moscow,Moscow,46.7307,-116.9986,United States,US,USA,Idaho,NaN,25579.0,1840019868


There are cities with the same names in many countries. Let's assume that if no country was said, it's probably the largest city.
One could in principle also use the administrative area name here but it doesn't seem as likely that people would put just the name of the administrative area but not the country or city name so let's try the city first.

In [159]:
world_cities = world_cities.sort_values(by="population").drop_duplicates(subset="city",keep="last").sort_index()

In [160]:
"""Again, take only interesting parts"""
city_to_country_code = world_cities[['city','iso3']]
city_to_country_code.city = city_to_country_code.city.str.lower()

In [161]:
clean_df["city_candidate"] = clean_df["location"].apply(lambda x: x.split(", ")[0].lower() if isinstance(x,str) else np.nan)

In [162]:
clean_df = clean_df.merge(city_to_country_code, left_on="city_candidate", right_on="city", how="left")
clean_df["COUNTRY_ALPHA3_CODE"][clean_df["COUNTRY_ALPHA3_CODE"].isnull()] = clean_df["iso3"][clean_df["COUNTRY_ALPHA3_CODE"].isnull()]

In [163]:
print("Proportion of matched: ", sum(clean_df.COUNTRY_ALPHA3_CODE.notnull())/sum(clean_df.location.notnull()))

Proportion of matched:  0.917169974115617


Seems like using the city name brings us up to 92.3% of matches.

In [164]:
"""Clean out some useless fields"""
clean_df.drop(["city_candidate", "city", "iso3","country"],axis=1, inplace=True)

In [165]:
"""See the remaining users that are still unmatched. Also prepare a lowercase location column for further analysis."""
clean_df["location_lc"] = clean_df.location.str.lower()
remaining = clean_df[clean_df.location.notnull()][clean_df.COUNTRY_ALPHA3_CODE.isnull()]
display(remaining[100:110])

,id,age,location,creation_date,reputation,up_votes,down_votes,COUNTRY_ALPHA3_CODE,location_lc
1267,3562541,NaN,Minnesota,2014-04-23 00:39:46.953 UTC,3,0,0,NaN,minnesota
1272,5749992,NaN,Somewhere Over the Rainbow,2016-01-05 23:47:53.463 UTC,3,0,0,NaN,somewhere over the rainbow
1280,8821565,NaN,"Taipei 101, Section 5, Xinyi Road, Xinyi Distr...",2017-10-23 20:07:19.247 UTC,3,0,0,NaN,"taipei 101, section 5, xinyi road, xinyi distr..."
1282,4366341,NaN,Reek,2014-12-16 12:33:32.24 UTC,3,0,0,NaN,reek
1286,4593809,NaN,"Hull, UK",2015-02-22 13:56:59.387 UTC,3,0,0,NaN,"hull, uk"
1287,7570131,NaN,"Walchand College of Engineering ,sangli-416415...",2017-02-15 16:22:29.603 UTC,3,0,0,NaN,"walchand college of engineering ,sangli-416415..."
1295,1792154,NaN,Idaho,2012-11-01 17:49:20.17 UTC,3,0,0,NaN,idaho
1296,2299708,NaN,UT,2013-04-19 14:55:46.24 UTC,3,0,0,NaN,ut
1303,2388579,NaN,Studio City,2013-05-16 06:00:49.067 UTC,3,0,0,NaN,studio city
1311,3330609,NaN,Online,2014-02-20 00:48:26.767 UTC,3,0,0,NaN,online


Seems like the remaining problem is in part that the format isn't necessarily
always city, admin area, country but sometimes some other ordering. Also, for instance UK is a common abbreviation for the United Kingdom but it wasn't in the country dataset. A regex search for the rest seems like a good idea.

In [166]:
country_name_to_code_2 = country_name_to_code.append([{"COUNTRY_NAME" : "uk", "COUNTRY_ALPHA3_CODE" : "GBR"},
                                                     {"COUNTRY_NAME" : "usa", "COUNTRY_ALPHA3_CODE" : "USA"},
                                                     {"COUNTRY_NAME" : "south korea", "COUNTRY_ALPHA3_CODE" : "KOR"}] ,ignore_index=True)

In [167]:
i = 0
for idx, row in country_name_to_code_2.iterrows():
    country_name,iso3 = row[0],row[1]
    matching = remaining.location_lc.str.contains("\\b"+country_name+"\\b")
    remaining.COUNTRY_ALPHA3_CODE[matching] = iso3
    if i % 1000 == 0:
        print(idx/len(country_name_to_code)*100, "% calculated")
    i += 1

0.0 % calculated
7.471049682480388 % calculated
14.942099364960775 % calculated
22.413149047441166 % calculated
29.88419872992155 % calculated
37.35524841240194 % calculated
44.82629809488233 % calculated
52.297347777362724 % calculated
59.7683974598431 % calculated
67.2394471423235 % calculated
74.71049682480388 % calculated
82.18154650728428 % calculated
89.65259618976467 % calculated
97.12364587224505 % calculated


In [168]:
clean_df = clean_df.merge(remaining.drop(["id","location","creation_date","reputation","up_votes","down_votes","location_lc","age"],axis=1), left_index=True, right_index=True, how="left")
clean_df.COUNTRY_ALPHA3_CODE_x[clean_df.COUNTRY_ALPHA3_CODE_x.isnull()] = clean_df.COUNTRY_ALPHA3_CODE_y[clean_df.COUNTRY_ALPHA3_CODE_x.isnull()]

In [169]:
clean_df.drop(["COUNTRY_ALPHA3_CODE_y"], axis=1, inplace=True)
clean_df.rename(columns={"COUNTRY_ALPHA3_CODE_x" : "country_iso3"}, inplace=True)

In [170]:
print("Proportion of matched: ", sum(clean_df.country_iso3.notnull())/sum(clean_df.location.notnull()))

Proportion of matched:  0.9437014667817084


Using the regex method brings the results already to 94.75 percent. Let's look at the remaining issues.

In [171]:
clean_df[clean_df.location.notnull() & clean_df.country_iso3.isnull()].head()

,id,age,location,creation_date,reputation,up_votes,down_votes,country_iso3,location_lc
15,8623520,NaN,deverloper,2017-09-17 19:52:14.057 UTC,1,0,0,NaN,deverloper
23,9946566,NaN,Korea,2018-06-15 12:18:19.137 UTC,1,0,0,NaN,korea
25,1003396,NaN,32960,2011-10-19 14:46:35.023 UTC,1,0,0,NaN,32960
29,2751922,NaN,"St. Louis, MO",2013-09-05 18:46:41.4 UTC,1,0,0,NaN,"st. louis, mo"
78,2712212,NaN,Montreal,2013-08-23 19:28:10.083 UTC,1,0,0,NaN,montreal


Seems like the remaining issues are mostly people giving out only the city or administrative area name in some format that wasn't considered before. Let's try to fix this with a similar regex matching for both.

In [172]:
world_cities_2 = pd.read_csv("data/worldcities.csv")
world_cities_2 = world_cities.sort_values(by="population").drop_duplicates(subset="admin_name",keep="last").sort_index()
admin_to_country_code = world_cities_2[['admin_name','iso3']]
admin_to_country_code.admin_name = admin_to_country_code.admin_name.str.lower()

In [173]:
"""Weird thing: NaN value in one of the admin names"""
remaining = clean_df[clean_df.location_lc.notnull()][clean_df.country_iso3.isnull()]
i = 0
for idx, row in admin_to_country_code.iterrows():
    admin_name,iso3 = row[0],row[1]
    matching = remaining.location_lc.str.contains("\\b"+str(admin_name)+"\\b")
    remaining.country_iso3[matching] = iso3
    if i % 1000 == 0:
        print(i/len(admin_to_country_code)*100, "% calculated")
    i += 1

0.0 % calculated
25.73340195573855 % calculated
51.4668039114771 % calculated
77.20020586721564 % calculated


In [174]:
remaining.rename(columns={"country_iso3" : "rem_iso3"}, inplace=True)

In [175]:
clean_df = clean_df.merge(remaining.drop(["id","location","creation_date","reputation","up_votes","down_votes","location_lc","age"],axis=1), left_index=True, right_index=True, how="left")
clean_df.country_iso3[clean_df.country_iso3.isnull()] = clean_df.rem_iso3[clean_df.country_iso3.isnull()]
clean_df.drop(["rem_iso3"], axis=1, inplace=True)

In [176]:
remaining = clean_df[clean_df.location_lc.notnull() & clean_df.country_iso3.isnull()]
i = 0
for idx, row in city_to_country_code.iterrows():
    city_name,iso3 = row[0],row[1]
    matching = remaining.location_lc.str.contains("\\b"+city_name+"\\b")
    remaining.country_iso3[matching] = iso3
    if i % 1000 == 0:
        print(i/len(city_to_country_code)*100, "% calculated")
    i += 1

0.0 % calculated
8.634832915983075 % calculated
17.26966583196615 % calculated
25.90449874794923 % calculated
34.5393316639323 % calculated
43.17416457991538 % calculated
51.80899749589846 % calculated
60.44383041188153 % calculated
69.0786633278646 % calculated
77.71349624384769 % calculated
86.34832915983075 % calculated
94.98316207581384 % calculated


In [177]:
remaining.rename(columns={"country_iso3" : "rem_iso3"}, inplace=True)

In [178]:
clean_df = clean_df.merge(remaining.drop(["id","location","creation_date","reputation","up_votes","down_votes","location_lc","age"],axis=1), left_index=True, right_index=True, how="left")
clean_df.country_iso3[clean_df.country_iso3.isnull()] = clean_df.rem_iso3[clean_df.country_iso3.isnull()]
clean_df.drop(["rem_iso3"], axis=1, inplace=True)

In [179]:
display(admin_to_country_code[[not isinstance(x,str) for x in admin_to_country_code.admin_name]])

,admin_name,iso3
7818,NaN,JEY


Funny fact: before we needed to have str(admin_name) in the code because there's a place called Nan in Thailand. 

In [180]:
sum(clean_df.country_iso3.notnull())/sum(clean_df.location.notnull())

0.9568593615185504

With the final method we get 95.66% of country matches to locations.

In [181]:
clean_df.drop(["location","location_lc"],axis=1,inplace=True)

In [182]:
clean_df.to_csv("data/stackoverflow_users_all_notext_clean.csv")

In [183]:
world_cities[world_cities.city.str.lower().str.contains("\\bada\\b")]

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
2010,Ada,Ada,45.8027,20.1285,Serbia,RS,SRB,Ada,admin,NaN,1688687750


There's also a city called Ada in Serbia. :)